In [ ]:
import sys
sys.path.append('../../../../Ecallisto/analisis/pycallisto/src/') 
import pycallisto as pyc
import os
import requests
import pycallisto_utils as utils

In [ ]:
def get_fit_file(year, month, day, hour, start_minute, end_minute, station):
    
    date= year+month+day
    #time='141500'
    
    for i in range(0,60,15):
        if i==0:
            limit_inf='00'
        else:
            limit_inf=str(i)
        limit_sup=str(i+15)

        if int(start_minute)>= int(limit_inf) and int(start_minute)<int(limit_sup):
            if int(end_minute) < int(limit_sup):
                time=hour+limit_inf+'00'
    
    red_flag=1
    found=0
    while(red_flag==1):
        URL = 'http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/'+year+'/'+month+'/'+day+'/'+station+'_'+date+'_'+time+'_59.fit.gz'
        # 2. download the data behind the URL
        response = requests.get(URL)

        # 3. Open the response into a new file
        fit_file=station+'_'+date+'_'+time+"_59.fit"
        open(fit_file, "wb").write(response.content)
        #f.close()
        
        try:
            for file in os.listdir():
                if(file==fit_file):
                    #Hay que guardar el mlc, spec, sub_spec
                    fit = pyc.PyCallisto.from_file(file)
                    #plot1 = fit.spectrogram(xtick=5, blevel=10)

                    #background_subtracted = fit.subtract_background()
                    #plot2 = background_subtracted.spectrogram()
                    
                    #background_subtracted.mean_light_curve(out_image ="Mean_light_curves/"+date+"_"+time+".png", grid=True)
            found=1
            red_flag=0
        except:
            new_second=time[4:]
            new_minute=time[2:4]
            new_hour=time[0:2]
            
            if int(time[4:])==0: # Second and minute change
                new_second='59'
                if int(time[2:4])==0: # Minute and hour change
                    new_minute='59'
                    if int(time[0:2])==0: 
                        new_hour='23'
                    elif int(time[0:2])<=10:
                        new_hour='0'+str(int(time[0:2])-1)
                    else:
                        new_hour=str(int(time[0:2])-1)
                else:
                    new_minute=str(int(time[2:4])-1)
            elif int(time[4])==5:
                new_second= str(int(time[4:])-1)
            
            if int(new_second)==50 or (int(time[4:])>0 and int(time[4:])<=9 ):
                if int(new_second)==50:
                    new_second='01'
                    if int(time[2:4])==59: # Minute and hour change
                        new_minute='00'
                        if int(time[0:2])==23:
                            new_hour='00'
                        elif int(time[0:2])<=8:
                            new_hour='0'+str(int(time[0:2])+1)
                        else:
                            new_hour=str(int(time[0:2])+1)
                    else:
                        new_minute='0'+str(int(time[2:4])+1)
                else:
                    new_second='0'+str(int(time[4:])+1)
                    
            if int(new_second)==10: # In case it doesn't find the fit file, to get out off the while loop
                red_flag=0
            os.remove(fit_file)
            time= new_hour+new_minute+new_second 
            
    if found==1:
        #os.remove(fit_file)
        open("FIT/"+fit_file, "wb").write(response.content)
        
    return found
             

In [ ]:
def get_txt_file(year, month, station):
    URL = 'http://soleil.i4ds.ch/solarradio/data/BurstLists/2010-yyyy_Monstein/'+year+'/e-CALLISTO_'+year+'_'+month+'.txt'
    # 2. download the data behind the URL
    response = requests.get(URL)

    # 3. Open the response into a new file
    txt_file=year+"_"+month+".txt"
    open(txt_file, "wb").write(response.content)
    
    f = open(txt_file, "r")
    lines=f.readlines()
    f.close()
    os.remove(txt_file)
    
    cont=0
    cont_2=0
    
    for line in lines:
        if line[0] != "#":
            row_info=line.split('\t')
            if len(row_info)>3:
                row_stations=row_info[3].split(', ')
                
                try:
                    row_stations.index(station)
                    day= row_info[0][6:8]
                    hour=row_info[1][0:2]
                    start_minute=row_info[1][3:5]
                    end_minute=row_info[1][9:]
                    if row_info[2]=='III':
                        cont_2=cont_2+1
                        cont= cont+get_fit_file(year, month, day, hour, start_minute, end_minute, station)
                except:
                    pass
    return [cont, cont_2]
                        

In [ ]:
station='HUMAIN'
year='2022'


for j in range(1,13):
    if j<10:
        month='0'+str(j)
    else:
        month=str(j)
    get_txt_file(year, month, station)
